Packages

In [62]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.linalg import lstsq

Loading in Datasets

In [15]:
gas_gaf = pd.read_csv("C:/Users/kshaz/Desktop/School Documents/Stony Brook Docs/Homework/AMS_595/Group Project/gas_gaf_595.csv")
neuro_20Y = pd.read_csv("C:/Users/kshaz/Desktop/School Documents/Stony Brook Docs/Homework/AMS_595/Group Project/neuro_20Y_595.csv")
neuro_25Y = pd.read_csv("C:/Users/kshaz/Desktop/School Documents/Stony Brook Docs/Homework/AMS_595/Group Project/neuro_25Y_595.csv")
neuro_27Y = pd.read_csv("C:/Users/kshaz/Desktop/School Documents/Stony Brook Docs/Homework/AMS_595/Group Project/neuro_27Y_595.csv")
stable = pd.read_csv("C:/Users/kshaz/Desktop/School Documents/Stony Brook Docs/Homework/AMS_595/Group Project/stable_595.csv")

Data Exploration

In [95]:
gas_gaf_20Y = gas_gaf[['ID','GRXX_APM_20_yr','GRXX_FPM_20_yr','GRXX_APY_20_yr','GRXX_FPY_20_yr']]
gas_gaf_25Y = gas_gaf[['ID','GRXXV_APM_25_yr','GRXXV_FPM_25_yr','GRXXV_APY_25_yr','GRXXV_FPY_25_yr']]
fiq_20Y = stable[['ID','fiq_strict_bartlett_20y_rescaled']]
fiq_25Y = stable[['ID','fiq_strict_bartlett_25y_rescaled']]

ggf_20Y = pd.merge(gas_gaf_20Y,fiq_20Y, on = 'ID')
ggf_25Y = pd.merge(gas_gaf_25Y,fiq_25Y, on = 'ID')

bins = [1,30,60,100]
category = [1,2,3]

ggf_20Y['GAS_PM_Category'] = pd.cut(ggf_20Y['GRXX_APM_20_yr'], bins=bins, labels=category, right=True, include_lowest=True)
ggf_20Y['GAS_PY_Category'] = pd.cut(ggf_20Y['GRXX_APY_20_yr'], bins=bins, labels=category, right=True, include_lowest=True)
ggf_20Y['GAF_PM_Category'] = pd.cut(ggf_20Y['GRXX_FPM_20_yr'], bins=bins, labels=category, right=True, include_lowest=True)
ggf_20Y['GAF_PY_Category'] = pd.cut(ggf_20Y['GRXX_FPY_20_yr'], bins=bins, labels=category, right=True, include_lowest=True)

ggf_25Y['GAS_PM_Category'] = pd.cut(ggf_25Y['GRXXV_APM_25_yr'], bins=bins, labels=category, right=True, include_lowest=True)
ggf_25Y['GAS_PY_Category'] = pd.cut(ggf_25Y['GRXXV_APY_25_yr'], bins=bins, labels=category, right=True, include_lowest=True)
ggf_25Y['GAF_PM_Category'] = pd.cut(ggf_25Y['GRXXV_FPM_25_yr'], bins=bins, labels=category, right=True, include_lowest=True)
ggf_25Y['GAF_PY_Category'] = pd.cut(ggf_25Y['GRXXV_FPY_25_yr'], bins=bins, labels=category, right=True, include_lowest=True)

Visualizing the relationship between IQ and GAS/GAF through scatterplots

In [ ]:
fig, sub = plt.subplots(2, 2, figsize=(10, 8))

sub[0,0].scatter(ggf_20Y['fiq_strict_bartlett_20y_rescaled'], ggf_20Y['GRXX_APM_20_yr'])
sub[0,0].set_title('IQ vs GAS for Past Month (20 Year Wave)')
sub[0,0].set_xlabel('IQ')
sub[0,0].set_ylabel('GAS')

sub[0,1].scatter(ggf_20Y['fiq_strict_bartlett_20y_rescaled'], ggf_20Y['GRXX_FPM_20_yr'])
sub[0,1].set_title('IQ vs GAS for Past Year (20 Year Wave)')
sub[0,1].set_xlabel('IQ')
sub[0,1].set_ylabel('GAF')

sub[1,0].scatter(ggf_20Y['fiq_strict_bartlett_20y_rescaled'], ggf_20Y['GRXX_APY_20_yr'])
sub[1,0].set_title('IQ vs GAF for Past Month (20 Year Wave)')
sub[1,0].set_xlabel('IQ')
sub[1,0].set_ylabel('GAS')

sub[1,1].scatter(ggf_20Y['fiq_strict_bartlett_20y_rescaled'], ggf_20Y['GRXX_FPY_20_yr'])
sub[1,1].set_title('IQ vs GAF for Past Year (20 Year Wave)')
sub[1,1].set_xlabel('IQ')
sub[1,1].set_ylabel('GAF')

plt.tight_layout()
plt.show()

Simple Linear Regression Model

In [ ]:
fiq_gas_py_20y = ggf_20Y[['fiq_strict_bartlett_20y_rescaled','GRXX_APY_20_yr']]
fiq_gas_py_20y_narm = ggf_20Y.dropna(subset=['fiq_strict_bartlett_20y_rescaled','GRXX_APY_20_yr'], how='any')

x = fiq_gas_py_20y_narm['fiq_strict_bartlett_20y_rescaled'].values.reshape(-1,1)
x_intercept = np.hstack((np.ones((x.shape[0], 1)), x))
y = fiq_gas_py_20y_narm['GRXX_APY_20_yr'].values

b = lstsq(x_intercept,y)
intercept,slope = b[0]

print('The intercept is:',intercept)
print('The slope is:',slope)

Linear Model Visualization

In [ ]:
regression_line = slope*x + intercept

plt.scatter(ggf_20Y['fiq_strict_bartlett_20y_rescaled'], ggf_20Y['GRXX_APY_20_yr'])
plt.plot(x, regression_line, color='red', label='Regression line')
plt.title('IQ vs GAF for Past Month (20 Year Wave)')
plt.xlabel('IQ')
plt.ylabel('GAS')

Priliminary Testing

In [129]:
cut_off_1 = (31-intercept)/slope # 62.59 -> rounded to 63
cut_off_2 = (61-intercept)/slope # 110.75 -> rounded to 111

bins_1 = [1,63,111,200]
category_1 = [1,2,3]

ggf_20Y['IQ_PY_Category'] = pd.cut(ggf_20Y['fiq_strict_bartlett_20y_rescaled'], bins=bins_1, labels=category_1, right=True, include_lowest=True)
ggf_20Y['Accuracy'] = (ggf_20Y['GAS_PY_Category'] == ggf_20Y['IQ_PY_Category']).astype(int)
ggf_20Y.loc[ggf_20Y[['GAS_PY_Category', 'IQ_PY_Category']].isnull().any(axis=1), 'Accuracy'] = np.nan 

ggf_20Y.Accuracy.value_counts() # correct = 141, incorrect = 100
 
print('Accuracy of Linear Model with Binning is:',141/(141+100))

Accuracy of Linear Model with Binning is: 0.5850622406639004
